In [1]:
import numpy as np
import pandas as pd
import sys
import os

In [2]:
pathToExcel = "E:\Github\Ladder-Iterative-Loadflow\Documentations\Datasets\IEEE 33 CDF (Updated)_2.xlsx"
tolerance = 10e-4

In [3]:
def checkIfAllTablesExist(arr):
    if "BUS DATA FOLLOWS" or "BRANCH DATA FOLLOWS" in arr:
        return True
    else:
        return False

In [4]:
def extractData(npArr, txt):

    lenOfArray = npArr.shape[0]
    widthOfArray = npArr.shape[1]
    forLoopStartValue = int(np.where(npArr==txt)[0])+1
    for i in range(forLoopStartValue, lenOfArray):
        if npArr[i][0] == -999:
            newArray = npArr[forLoopStartValue:i]
            break

    if txt == "BUS DATA FOLLOWS":
        # Add code here for extracting SBase
        arrayToReturn = np.zeros((newArray.shape[0],4))
        arrayToReturn[:,0] = newArray[:,0]
        arrayToReturn[:,1] = newArray[:,7]
        arrayToReturn[:,2] = newArray[:,8]
        arrayToReturn[:,3] = newArray[:,11]
    elif txt == "BRANCH DATA FOLLOWS":
        arrayToReturn = np.zeros((newArray.shape[0],4))
        arrayToReturn[:,0] = newArray[:,0]
        arrayToReturn[:,1] = newArray[:,1]
        arrayToReturn[:,2] = newArray[:,6]
        arrayToReturn[:,3] = newArray[:,7]
        
    
    return arrayToReturn


In [5]:
def branchReorder(branchData):
    newData = branchData
    lenOfArray = newData.shape[0]
    zerosCol = np.zeros((newData.shape[0],1))
    newData = np.c_[newData, zerosCol]
    newBranchDataArr = np.zeros((newData.shape[0],4))
    n=0
    duplicateRan = False
    for i in range(0, lenOfArray):
        dup = np.where(newData[:,0]==newData[i,0])
        if len(dup[0])==1 and (newData[i,4]==0):
            newBranchDataArr[n,0] = newData[i,0]
            newBranchDataArr[n,1] = newData[i,1]
            newBranchDataArr[n,2] = newData[i,2]
            newBranchDataArr[n,3] = newData[i,3]
            newData[i,4] = 1
            n=n+1            
        else:
            for j in dup[0]:
                k = j
                if (j > i) and (newData[k,4]==0):
                    
                    while(1):
                        if k+1 == lenOfArray:
                            if (newData[k,0]==newData[k-1,1]):
                                newBranchDataArr[n,0] = newData[k,0]
                                newBranchDataArr[n,1] = newData[k,1]
                                newBranchDataArr[n,2] = newData[k,2]
                                newBranchDataArr[n,3] = newData[k,3]
                                newData[k,4] = 1
                                n = n+1
                                k = k+1
                            break
                        if (newData[k,1]==newData[k+1,0]):
                            newBranchDataArr[n,0] = newData[k,0]
                            newBranchDataArr[n,1] = newData[k,1]
                            newBranchDataArr[n,2] = newData[k,2]
                            newBranchDataArr[n,3] = newData[k,3]
                            newData[k,4] = 1
                            n = n+1
                            k = k+1
                        elif (newData[k,1]!=newData[k+1,0]) and (newBranchDataArr[n-1,0] != newData[k,0]):
                            newBranchDataArr[n,0] = newData[k,0]
                            newBranchDataArr[n,1] = newData[k,1]
                            newBranchDataArr[n,2] = newData[k,2]
                            newBranchDataArr[n,3] = newData[k,3]
                            newData[k,4] = 1
                            n = n+1
                            k = k+1
                            break
                    duplicateRan = True
        if duplicateRan == True and (newData[i,4]==0):
            newBranchDataArr[n,0] = newData[i,0]
            newBranchDataArr[n,1] = newData[i,1]
            newBranchDataArr[n,2] = newData[i,2]
            newBranchDataArr[n,3] = newData[i,3]
            newData[i,4] = 1
            n=n+1
            duplicateRan = False
    return newBranchDataArr

In [6]:
#def dataParser(path):
path = pathToExcel
readInput = pd.read_excel(path, header=None)
numpyConversion = readInput.to_numpy()
checker = checkIfAllTablesExist(readInput)

if checker == False:
    raise ValueError("Excel does not contain Bus or Branch data, please double check and try again")

busData = extractData(numpyConversion, "BUS DATA FOLLOWS")
branchData = extractData(numpyConversion, "BRANCH DATA FOLLOWS")

sortedBranchData = branchReorder(branchData)

# To display
x = pd.DataFrame(busData)
display(x)




,0,1,2,3
0,1.0,0.000,0.000,12.66
1,2.0,0.100,0.060,12.66
2,3.0,0.090,0.040,12.66
3,4.0,0.120,0.080,12.66
4,5.0,0.060,0.030,12.66
5,6.0,0.060,0.020,12.66
6,7.0,0.200,0.100,12.66
7,8.0,0.200,0.100,12.66
8,9.0,0.060,0.020,12.66
9,10.0,0.060,0.020,12.66
